In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter as cnt
import string
from nltk.stem.porter import PorterStemmer
import nltk
from sklearn.metrics.pairwise import linear_kernel
import csv

In [2]:
dev=pd.read_csv('data/devSet.csv')
dev.shape

(388800, 6)

In [3]:
q1=list(dev.question1)
q2=list(dev.question2)
dup=list(dev.is_duplicate)

In [4]:
token_dict = {}
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [5]:
count=0
for text1, text2 in zip(q1, q2):
    if text1 is np.nan: continue
    if text2 is np.nan: continue
    if len(text1) == 0: continue
    if len(text2) == 0: continue
    count=count+1 #for q1 query id
    token_dict[count] = text1
    count=count+1 # for q2 query id
    token_dict[count] = text2

In [6]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english',
                       strip_accents='ascii')
tfs = tfidf.fit_transform(token_dict.values())

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wo

In [16]:
count=0
dup_count=0
correct=0
for d in dup:
    q1=tfs[count:count+1]
    count=count+1
    q2=tfs[count:count+1]
    count=count+1
    
    if (q1).shape[0] < 1:
        continue
    if (q2).shape[0] < 1:
        continue
    cosine_similarities = linear_kernel(q1, q2).flatten()
    if d == 1:
        dup_count=dup_count+1
        if cosine_similarities[0] > 0.5:
            correct=correct+1

In [17]:
precision=correct/dup_count
print('Precision = {}'.format(precision))

Precision = 0.6543577581697066


In [18]:
test=pd.read_csv('data/test.csv')

In [19]:
results_test=[]
def calculate_cosine_similarity(x):
    global tfidf
    global results_test
    id=x[0]
    qid1=x[1]
    qid2=x[2]
    question1=x[3]
    question2=x[4]
    
    if question1 is np.nan:
        results_test.append((id, 0))
        return
    if question2 is np.nan:
        results_test.append((id, 0))
        return
    if len(question1) == 0:
        results_test.append((id, 0))
        return
    if len(question2) == 0:
        results_test.append((id, 0))
        return 
    
    q1_vector= tfidf.transform([question1])
    q2_vector= tfidf.transform([question2])
    cosine_similarities = linear_kernel(q1_vector, q2_vector).flatten()
    if cosine_similarities[0] > 0.5:
        results_test.append((id, 1))
    else:
        results_test.append((id, 0))
test.apply(lambda x: calculate_cosine_similarity(x), axis=1)
print('done')

done


In [20]:
len(results_test)

15490

In [21]:
15490

15490

In [23]:
result_np=np.array(results_test)
pd.DataFrame(result_np).to_csv("test.csv",header=['id', 'is_duplicate'],index=False)

In [14]:
(tfs[0:1]).shape[0]

1